In [6]:

#Ejercicio 3
import numpy as np

class LinearSystemSolver:
    def __init__(self, A, b):
        self.A = A
        self.b = b
        self.x0 = np.zeros_like(b) 

    def jacobi(self, tol=1e-10, max_iterations=1000):
        n = len(self.b)
        x = np.zeros_like(self.b)
        D = np.diag(np.diag(self.A))
        R = self.A - D
        iterations = 0
        
        for _ in range(max_iterations):
            x_new = (self.b - np.dot(R, x)) / np.diag(self.A)
            iterations += 1
            if np.linalg.norm(x_new - x, ord=np.inf) < tol:
                print(f"Jacobi: Converged in {iterations} iterations.")
                return x_new
            x = x_new
        
        print(f"Jacobi: Maximum iterations reached ({iterations}).")
        return x

    def gauss_seidel(self, tol=1e-10, max_iterations=1000):
        n = len(self.b)
        x = np.zeros_like(self.b)
        iterations = 0

        for _ in range(max_iterations):
            x_new = np.copy(x)
            for i in range(n):
                sum1 = np.dot(self.A[i, :i], x_new[:i])
                sum2 = np.dot(self.A[i, i+1:], x[i+1:])
                x_new[i] = (self.b[i] - sum1 - sum2) / self.A[i, i]
            iterations += 1
            if np.linalg.norm(x_new - x, ord=np.inf) < tol:
                print(f"Gauss-Seidel: Converged in {iterations} iterations.")
                return x_new
            x = x_new
        
        print(f"Gauss-Seidel: Maximum iterations reached ({iterations}).")
        return x

# Ejemplo de prueba
A = np.array([[4, -1, 0, 0], [-1, 4, -1, 0], [0, -1, 4, -1], [0, 0, -1, 3]])
b = np.array([15, 10, 10, 10])

solver = LinearSystemSolver(A, b)
jacobi_solution = solver.jacobi()
gauss_seidel_solution = solver.gauss_seidel()

print("Solución Jacobi:", jacobi_solution)
print("Solución Gauss-Seidel:", gauss_seidel_solution)



Jacobi: Converged in 30 iterations.
Gauss-Seidel: Converged in 3 iterations.
Solución Jacobi: [5. 5. 5. 5.]
Solución Gauss-Seidel: [4 4 4 4]


In [7]:
#Ejercicio 4
def matrix_multiply(A, B):
    # Dimensiones de las matrices
    rows_A = len(A)
    cols_A = len(A[0])
    rows_B = len(B)
    cols_B = len(B[0])
    
    # Verificar si las matrices se pueden multiplicar
    if cols_A != rows_B:
        raise ValueError("El número de columnas de A debe ser igual al número de filas de B.")
    
    # Crear una matriz resultado inicializada en ceros
    result = [[0 for _ in range(cols_B)] for _ in range(rows_A)]
    
    for i in range(rows_A):
        for j in range(cols_B):
            for k in range(cols_A):
                result[i][j] += A[i][k] * B[k][j]
    
    return result

A = [[1, 0, 0], 
     [5, 1, 0], 
     [-2, 3, 1]]

B = [[4, -2, 1], 
     [0, 3, 7], 
     [0, 0, 2]]

result_matrix = matrix_multiply(A, B)

for row in result_matrix:
    print(row)


[4, -2, 1]
[20, -7, 12]
[-8, 13, 21]


In [8]:
#Ejercicio 8
def lu_decomposition(A):
    n = len(A)
    L = [[0.0 for _ in range(n)] for _ in range(n)]
    U = [[0.0 for _ in range(n)] for _ in range(n)]
    
    
    for i in range(n):
        # Calcular la matriz U
        for k in range(i, n):
            sum = 0
            for j in range(i):
                sum += L[i][j] * U[j][k]
            U[i][k] = A[i][k] - sum
        
        # Calcular la matriz L
        for k in range(i, n):
            if i == k:
                L[i][i] = 1  
            else:
                sum = 0
                for j in range(i):
                    sum += L[k][j] * U[j][i]
                L[k][i] = (A[k][i] - sum) / U[i][i]
    
    return L, U


A = [[4, -2, 1],
     [20, -7, 12],
     [-8, 13, 17]]


L, U = lu_decomposition(A)


print("Matriz L:")
for row in L:
    print(row)

print("\nMatriz U:")
for row in U:
    print(row)


Matriz L:
[1, 0.0, 0.0]
[5.0, 1, 0.0]
[-2.0, 3.0, 1]

Matriz U:
[4, -2, 1]
[0.0, 3.0, 7.0]
[0.0, 0.0, -2.0]


In [ ]:
#Ejercicio 10 
import numpy as np
from math import atan2, cos, sin, pi

# Definir la matriz A
A = np.array([[4, 1, 1],
              [1, 3, 2],
              [1, 2, 5]])

def max_off_diagonal(A):
    # Encuentra el valor máximo fuera de la diagonal
    n = A.shape[0]
    max_val = 0
    p, q = 0, 0
    for i in range(n):
        for j in range(i+1, n):
            if abs(A[i, j]) > abs(max_val):
                max_val = A[i, j]
                p, q = i, j
    return p, q, max_val

def jacobi_rotation(A, tol=1e-10):
    n = A.shape[0]
    V = np.eye(n)  # Matriz de autovectores
    iterations = 0
    while True:
        p, q, max_val = max_off_diagonal(A)
        if abs(max_val) < tol:
            break
        
        # Calcular el ángulo theta
        if A[p, p] == A[q, q]:
            theta = pi / 4
        else:
            theta = 0.5 * atan2(2 * A[p, q], A[q, q] - A[p, p])

        cos_theta = cos(theta)
        sin_theta = sin(theta)

        # Crear la matriz de rotación
        R = np.eye(n)
        R[p, p] = cos_theta
        R[q, q] = cos_theta
        R[p, q] = -sin_theta
        R[q, p] = sin_theta

        # Actualizar A y V
        A = np.dot(R.T, np.dot(A, R))
        V = np.dot(V, R)
        
        iterations += 1

    eigenvalues = np.diag(A)
    eigenvectors = V
    return eigenvalues, eigenvectors, iterations

# Aplicar el método de Jacobi
eigenvalues_jacobi, eigenvectors_jacobi, iters = jacobi_rotation(np.copy(A))

# Comparar con la función de Numpy
eigenvalues_numpy, eigenvectors_numpy = np.linalg.eig(A)

print("Valores propios usando Jacobi:", eigenvalues_jacobi)
print("Vectores propios usando Jacobi:\n", eigenvectors_jacobi)
print("Número de iteraciones:", iters)

print("\nValores propios usando Numpy:", eigenvalues_numpy)
print("Vectores propios usando Numpy:\n", eigenvectors_numpy)


In [10]:
#Ejercicio 12
import numpy as np

def f(x):
    x1, x2, x3 = x
    return np.array([
        6*x1 - 2*np.cos(x2*x3) - 1,
        9*x2 + np.sqrt(x1**2 + np.sin(x3) + 1.06 + 0.9),
        60*x3 + 3*np.exp(x1*x2) + 10*np.pi - 3
    ])

def jacobian(x):
    x1, x2, x3 = x
    return np.array([
        [6, 2*x3*np.sin(x2*x3), 2*x2*np.sin(x2*x3)],
        [x1/np.sqrt(x1**2 + np.sin(x3) + 1.96), 9, np.cos(x3)/(2*np.sqrt(x1**2 + np.sin(x3) + 1.96))],
        [3*x2*np.exp(x1*x2), 3*x1*np.exp(x1*x2), 60]
    ])

def newton_raphson(x0, tol=1e-6, max_iter=100):
    x = x0
    for i in range(max_iter):
        fx = f(x)
        if np.linalg.norm(fx) < tol:
            return x, i
        J = jacobian(x)
        delta = np.linalg.solve(J, -fx)
        x = x + delta
    return x, max_iter

x0 = np.array([0, 0, 0])
solution, iterations = newton_raphson(x0)

print(f"Solución: x1 = {solution[0]:.6f}, x2 = {solution[1]:.6f}, x3 = {solution[2]:.6f}")
print(f"Iteraciones: {iterations}")
print(f"Error: {np.linalg.norm(f(solution)):.6e}")

Solución: x1 = 0.499048, x2 = -0.145385, x3 = -0.520100
Iteraciones: 3
Error: 3.393938e-12


In [12]:
#Ejercicio 13
import numpy as np

def f(x):
    x1, x2, x3 = x
    return np.array([
        6*x1 - 2*np.cos(x2*x3) - 1,
        9*x2 + np.sqrt(x1**2 + np.sin(x3) + 1.06 + 0.9),
        60*x3 + 3*np.exp(x1*x2) + 10*np.pi - 3
    ])

def jacobian_fourth_order(func, x, h=1e-5):
    n = len(x)
    f = func(x)
    m = len(f)
    J = np.zeros((m, n))
    
    for j in range(n):
        x_plus_2h = x.copy()
        x_plus_2h[j] += 2*h
        
        x_plus_h = x.copy()
        x_plus_h[j] += h
        
        x_minus_h = x.copy()
        x_minus_h[j] -= h
        
        x_minus_2h = x.copy()
        x_minus_2h[j] -= 2*h
        
        f_plus_2h = func(x_plus_2h)
        f_plus_h = func(x_plus_h)
        f_minus_h = func(x_minus_h)
        f_minus_2h = func(x_minus_2h)
        
        J[:, j] = (-f_plus_2h + 8*f_plus_h - 8*f_minus_h + f_minus_2h) / (12*h)
    
    return J

def jacobian_second_order(func, x, h=1e-5):
    n = len(x)
    f = func(x)
    m = len(f)
    J = np.zeros((m, n))
    
    for j in range(n):
        x_plus_h = x.copy()
        x_plus_h[j] += h
        
        x_minus_h = x.copy()
        x_minus_h[j] -= h
        
        f_plus_h = func(x_plus_h)
        f_minus_h = func(x_minus_h)
        
        J[:, j] = (f_plus_h - f_minus_h) / (2*h)
    
    return J

def analytical_jacobian(x):
    x1, x2, x3 = x
    return np.array([
        [6, 2*x3*np.sin(x2*x3), 2*x2*np.sin(x2*x3)],
        [x1/np.sqrt(x1**2 + np.sin(x3) + 1.96), 9, np.cos(x3)/(2*np.sqrt(x1**2 + np.sin(x3) + 1.96))],
        [3*x2*np.exp(x1*x2), 3*x1*np.exp(x1*x2), 60]
    ])

# Punto de evaluación
x = np.array([0.5, 0.5, 0.5])
h = 0.01

J_analytical = analytical_jacobian(x)
J_fourth_order = jacobian_fourth_order(f, x, h)
J_second_order = jacobian_second_order(f, x, h)

print("Jacobiano analítico:")
print(J_analytical)
print("\nJacobiano estimado (cuarto orden):")
print(J_fourth_order)
print("\nJacobiano estimado (segundo orden):")
print(J_second_order)

# Cálculo de errores
error_fourth = np.linalg.norm(J_analytical - J_fourth_order)
error_second = np.linalg.norm(J_analytical - J_second_order)

print(f"\nError (cuarto orden): {error_fourth}")
print(f"Error (segundo orden): {error_second}")

# Búsqueda del h para igualar precisión
h_second = h
while True:
    h_second = h_second / 2
    J_second_order = jacobian_second_order(f, x, h_second)
    error_second = np.linalg.norm(J_analytical - J_second_order)
    if error_second <= error_fourth:
        break

print(f"\nValor de h para igualar precisión: {h_second}")

Jacobiano analítico:
[[ 6.          0.24740396  0.24740396]
 [ 0.30488794  9.          0.26756434]
 [ 1.92603813  1.92603813 60.        ]]

Jacobiano estimado (cuarto orden):
[[ 6.          0.24740396  0.24740396]
 [ 0.30488794  9.          0.26756434]
 [ 1.92603812  1.92603812 60.        ]]

Jacobiano estimado (segundo orden):
[[ 6.          0.24740293  0.24740293]
 [ 0.3048828   9.          0.26756143]
 [ 1.92604615  1.92604615 60.        ]]

Error (cuarto orden): 4.5372353552698125e-10
Error (segundo orden): 1.2877797771612658e-05

Valor de h para igualar precisión: 3.90625e-05


In [18]:
#Ejercicio 14
import sympy as sp

# Definimos las matrices generadoras
J1 = sp.Matrix([[0, 0, 0], [0, 0, -1], [0, 1, 0]])
J2 = sp.Matrix([[0, 0, 1], [0, 0, 0], [-1, 0, 0]])
J3 = sp.Matrix([[0, -1, 0], [1, 0, 0], [0, 0, 0]])

# Función para calcular el conmutador
def commutator(A, B):
    return A*B - B*A

# Función para obtener el símbolo de Levi-Civita
def levi_civita(i, j, k):
    if (i, j, k) in [(1,2,3), (2,3,1), (3,1,2)]:
        return 1
    elif (i, j, k) in [(3,2,1), (1,3,2), (2,1,3)]:
        return -1
    else:
        return 0

# Lista de generadores
generators = [J1, J2, J3]

# Verificamos las relaciones de conmutación
for i in range(3):
    for j in range(3):
        comm = commutator(generators[i], generators[j])
        expected = sp.zeros(3, 3)
        for k in range(3):
            expected += levi_civita(i+1, j+1, k+1) * generators[k]
        
        print(f"[J{i+1}, J{j+1}] =")
        sp.pprint(comm)
        print(f"ε{i+1}{j+1}k Jk =")
        sp.pprint(expected)
        print("¿Son iguales?", comm == expected)
        print()

# Verificamos la antisimetría del conmutador
print("Verificación de la antisimetría del conmutador:")
for i in range(3):
    for j in range(3):
        comm_ij = commutator(generators[i], generators[j])
        comm_ji = commutator(generators[j], generators[i])
        print(f"[J{i+1}, J{j+1}] = -[J{j+1}, J{i+1}]: {comm_ij == -comm_ji}")

# Verificamos la identidad de Jacobi
print("\nVerificación de la identidad de Jacobi:")
for i in range(3):
    for j in range(3):
        for k in range(3):
            jacobi = (commutator(generators[i], commutator(generators[j], generators[k])) +
                      commutator(generators[j], commutator(generators[k], generators[i])) +
                      commutator(generators[k], commutator(generators[i], generators[j])))
            print(f"[[J{i+1}, J{j+1}], J{k+1}] + [[J{j+1}, J{k+1}], J{i+1}] + [[J{k+1}, J{i+1}], J{j+1}] = 0: {jacobi == sp.zeros(3,3)}")

[J1, J1] =
⎡0  0  0⎤
⎢       ⎥
⎢0  0  0⎥
⎢       ⎥
⎣0  0  0⎦
ε11k Jk =
⎡0  0  0⎤
⎢       ⎥
⎢0  0  0⎥
⎢       ⎥
⎣0  0  0⎦
¿Son iguales? True

[J1, J2] =
⎡0  -1  0⎤
⎢        ⎥
⎢1  0   0⎥
⎢        ⎥
⎣0  0   0⎦
ε12k Jk =
⎡0  -1  0⎤
⎢        ⎥
⎢1  0   0⎥
⎢        ⎥
⎣0  0   0⎦
¿Son iguales? True

[J1, J3] =
⎡0  0  -1⎤
⎢        ⎥
⎢0  0  0 ⎥
⎢        ⎥
⎣1  0  0 ⎦
ε13k Jk =
⎡0  0  -1⎤
⎢        ⎥
⎢0  0  0 ⎥
⎢        ⎥
⎣1  0  0 ⎦
¿Son iguales? True

[J2, J1] =
⎡0   1  0⎤
⎢        ⎥
⎢-1  0  0⎥
⎢        ⎥
⎣0   0  0⎦
ε21k Jk =
⎡0   1  0⎤
⎢        ⎥
⎢-1  0  0⎥
⎢        ⎥
⎣0   0  0⎦
¿Son iguales? True

[J2, J2] =
⎡0  0  0⎤
⎢       ⎥
⎢0  0  0⎥
⎢       ⎥
⎣0  0  0⎦
ε22k Jk =
⎡0  0  0⎤
⎢       ⎥
⎢0  0  0⎥
⎢       ⎥
⎣0  0  0⎦
¿Son iguales? True

[J2, J3] =
⎡0  0  0 ⎤
⎢        ⎥
⎢0  0  -1⎥
⎢        ⎥
⎣0  1  0 ⎦
ε23k Jk =
⎡0  0  0 ⎤
⎢        ⎥
⎢0  0  -1⎥
⎢        ⎥
⎣0  1  0 ⎦
¿Son iguales? True

[J3, J1] =
⎡0   0  1⎤
⎢        ⎥
⎢0   0  0⎥
⎢        ⎥
⎣-1  0  0⎦
ε31k Jk =
⎡0   0  1⎤
⎢        ⎥
⎢0   0  0⎥
⎢    